# 시나리오
- 교차검증을 통해 최적 파라미터 값을 찾을 수 있음
- 결정트리 모델의 최적의 파라미터를 찾으려고 함
- 결정트리 모델에서 최적의 max_depth를 찾고 min_samples_split을 변경하면서 최적 값을 찾을 수 없음
- 하나의 파라미터가 변경되면 다른 파라미터의 결과도 고정되지 않음(연관 변수를 함께 변경하면서 찾아야 함)
- 모델마다 하이퍼 파라미터의 갯수가 다름(1개 ~ 6개 등 모델마다 다양)
- 수동으로 일일이 파라미터를 변경하면서 모델의 점수를 확인하는 것은 매우 힘듦


- 사이킷런의 GridSearchCV를 사용하면 최적의 하이퍼파라미터를 찾을 수 있음(교차 검증 내장)

## 하이퍼 파리미터 튜닝(Hyper Parameter Tuning)
- 머신러닝 모델이 학습하는 파라미터는 모델 파라미터
- 사용자가 지정해야 하는 파라미터는 하이퍼파라미터
- 모든 하이퍼파라미터는 클래스 또는 매개변수로 표현

### 데이터 준비 및 나누기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

wine = pd.read_csv('/content/drive/MyDrive/NEW/2week/wine_csv_data.csv')

# class열은 종속변수(y), 나머지는 독립변수(X)
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

#### GridSearchCV 정의
- 결정트리 모델에서 min_impurity_decrease 파라미터 리스트 준비
- 파라미터 리스트를 이용하여 GridSearchCV객체 생성
- GridSearchCV의 cv 디폴트 값은 5(총 5번 수행. 총 25번 진행)
- n_jobs는 병렬실행에 사용할 CPU코어 수 지정(기본값은 1. -1로 지정하면 모든 코어 사용)

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
          'max_depth': [x for x in range(1,40,1)]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
dt = gs.best_estimator_
dt.score(X_train, y_train)

0.9615162593804117

In [17]:
gs.best_params_

{'max_depth': 20, 'min_impurity_decrease': 0.0001}

##### 결과
- 훈련이 끝나면 25개의 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트로 다시 훈련한 모델이 준비됨
- best_estimator_로 사용 가능
- best_params_는 가장 좋은 하이퍼 파라미터값이 저장되어 있음

#### 각 매개변수에서 수행한 교차 검증의 평균 점수 확인

In [18]:
gs.cv_results_['mean_test_score']

array([0.75793737, 0.75793737, 0.75793737, 0.75793737, 0.75793737,
       0.81393555, 0.81393555, 0.81393555, 0.81393555, 0.81393555,
       0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.85337806, 0.85337806, 0.85337806, 0.85337806, 0.85318557,
       0.85780355, 0.85799604, 0.85857352, 0.85857352, 0.85838102,
       0.85645721, 0.85799678, 0.85876675, 0.85972866, 0.86088306,
       0.85607093, 0.85761031, 0.85799511, 0.85991893, 0.86280466,
       0.86088343, 0.86242226, 0.86242208, 0.86242282, 0.86492374,
       0.86222792, 0.86376749, 0.86357574, 0.86550048, 0.86665414,
       0.86088362, 0.86299956, 0.86126731, 0.8662686 , 0.86723088,
       0.86319112, 0.86511439, 0.86511605, 0.86819353, 0.86761605,
       0.86646313, 0.86511513, 0.86550011, 0.86723144, 0.86761605,
       0.86588584, 0.86492189, 0.86434497, 0.86742393, 0.86761605,
       0.86357611, 0.86761531, 0.86492226, 0.86780891, 0.86761605,
       0.86472866, 0.86453691, 0.86472977, 0.86780891, 0.86761

#### 평균 점수에서 가장 큰 값 구하기
- gs.best_params_ 값과 동일함

In [20]:
import numpy as np

best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'max_depth': 20, 'min_impurity_decrease': 0.0001}

### 좀 더 복잡한 매개변수 조합 탐색

In [ ]:
range(2, 100, 10)

range(2, 100, 10)

In [21]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

- 교차 검증 횟수
    - 9 x 15 x 10 = 1350개
- 여기에 기본 5폴드
    - 6750개

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [24]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

##### 생각해볼 것
- 탐색할 매개변수의 간격
- 예제에서 0.0001부터 시작하는 간격은 근거가 없음(더 넓거나 좁게 설정 가능함)

### 랜덤 서치
- RandomizedSearchCV
- 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음
- 매개변수를 샘플링할 수 있는 확률 분포 객체를 이용

#### 확률 분포 클래스 활용
- 주어진 범위에서 고르게 값을 선택(균등 분포 샘플링)
- uniform(): 실수값
- randint(): 정수값

#### scipy 사용
- 매개변수 값이 수치형(연속적)
- scipy에 확률 분포 객체를 전달하여 특정 범위 내에서 지정된 횟수만큼 값을 샘플링하고 교차검증까지 가능

In [25]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 9, 9, 6, 3, 9, 5, 7, 2, 5])

In [26]:
# 샘플링 갯수를 늘려서 고르게 가져오는 것 확인
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110,  89,  99,  96,  92, 108, 104, 101,  88, 113]))

##### 결과
- 샘플링 횟수는 시스템 자원이 허락하는 범위에서 최대한 큰 것이 유리

In [ ]:
# 0 ~ 1 사이의 10개 실수를 추출
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.25022293, 0.29121426, 0.86938672, 0.74434921, 0.37515348,
       0.72034619, 0.9969255 , 0.88386968, 0.56306274, 0.49074374])

##### min_samples_leaf 매개변수 사용
- min_samples_leaf는 leaf노드가 되기 위한 최소 샘플의 갯수
- 어떤 노드를 분할해서 만들어지는 자식 노드들의 샘플 수가 이 값보다 작으면 분할하지 않흠

In [27]:
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d3ca35b1300>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d3ca8133af0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d3ca35b3490>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d3ca35b0910>},
                   random_state=42)

In [28]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [29]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

#### 훈련세트로 훈련된 객체를 이용하여 테스트 데이터셋점수 확인

##### 해석
- 최적 모델(best_estimator_)는 이미 훈련세트로 훈련되어 best_estimator_에 저장되어 있음

In [ ]:
dt = gs.best_estimator_

dt.score(X_test, y_test)

0.86

##### 결과
- 테스트데이터셋의 점수는 검증 세트에 대한 점수보다 약간 작은 것이 일반적
- 다양한 매개변수(하이퍼 파라미터)를 이용하여 여러 번 테스트 하는 작업을 GridSearchCV/RandomizedSearchCV 적용

### 정리
- 교차 검증을 이용하여 다양한 파라미터를 탐색
- 그리스서치 관련 클래스를 활용하여 자동화 가능